# Evaluate trained models on a task and check success rate 

Load the necessary library

In [1]:
import numpy as np
import torch
import hydra
from omegaconf import DictConfig
import datetime
from omniisaacgymenvs.utils.hydra_cfg.hydra_utils import *
from omniisaacgymenvs.utils.hydra_cfg.reformat import omegaconf_to_dict, print_dict
from rl_games.algos_torch.players import PpoPlayerDiscrete
from rl_games.algos_torch.players import BasicPpoPlayerContinuous, BasicPpoPlayerDiscrete

from omniisaacgymenvs.utils.rlgames.rlgames_utils import RLGPUAlgoObserver, RLGPUEnv

from omniisaacgymenvs.scripts.rlgames_train import RLGTrainer
from rl_games.torch_runner import Runner
from omniisaacgymenvs.utils.task_util import initialize_task
from omniisaacgymenvs.envs.vec_env_rlgames import VecEnvRLGames
from omniisaacgymenvs.utils.config_utils.path_utils import retrieve_checkpoint_path
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from torch._C import fork
from gym import spaces
import numpy as np
import torch
import yaml
import os
import glob

from plot_experiment import plot_episode_data_virtual
from eval_metrics import success_rate_from_distances

In [2]:
# specify the experiment load directory
load_dir = "../new_mass/"
experiments = os.listdir(load_dir)
print(f'Experiments found in {load_dir} folder: {len(experiments)}')

Experiments found in ../new_mass/ folder: 20


In [3]:
# filter out invalid experiments and retrieve valid models
def get_valid_models(load_dir, experiment):
    valid_models = []
    invalid_experiments = []
    for experiment in experiments:
        try:
            file_pattern = os.path.join(load_dir, experiment, "nn", "last_*ep_2000_rew__*.pth")
            model = glob.glob(file_pattern)
            if model:
                valid_models.append(model[0])
        except:
            invalid_experiments.append(experiment)
    if invalid_experiments:
        print(f'Invalid experiments: {invalid_experiments}')
    else:
        print('All experiments are valid')
    return valid_models

In [4]:
models = get_valid_models(load_dir, experiments)
if not models:
    print('No valid models found')
    exit()

All experiments are valid


In [5]:
models[:2]

['../new_mass/MLP_GTXY_UF_0.25_ST_PE_0.01_PAV_0.5_PLV_0.05_5kg/nn/last_MLP_GTXY_UF_0.25_ST_PE_0.01_PAV_0.5_PLV_0.05_5kg_ep_2000_rew__597.8011_.pth',
 '../new_mass/MLP_GTXY_UF_0.25_ST_PE_0.05_PAV_1.5_PLV_0.01_5kg/nn/last_MLP_GTXY_UF_0.25_ST_PE_0.05_PAV_1.5_PLV_0.01_5kg_ep_2000_rew__597.8011_.pth']

In [6]:

def eval_multi_agents(agent, models, horizon):

    base_dir = "./evaluations/"
    experiment_name = models[0].split("/")[1]
    print(f'Experiment name: {experiment_name}')
    evaluation_dir = base_dir + experiment_name + "/"
    os.makedirs(evaluation_dir, exist_ok=True)

    agent.restore(models[0])

    store_all_agents = True # store all agents generated data, if false only the first agent is stored
    is_done = False
    env = agent.env
    obs = env.reset()

    ep_data = {'act': [], 'obs': [], 'rews': [], 'info': [], 'all_dist': []}
    total_reward = 0
    num_steps = 0
    
    total_num_steps = 800
    for _ in range(total_num_steps):
        actions = agent.get_action(obs['obs'], is_deterministic=True)
        obs, reward, done, info = env.step(actions)
        
        #print(f'Step {num_steps}: obs={obs["obs"]}, rews={reward}, dones={done}, info={info} \n')
        if store_all_agents:
            ep_data['act'].append(actions.cpu().numpy())
            ep_data['obs'].append(obs['obs']['state'].cpu().numpy())
            ep_data['rews'].append(reward.cpu().numpy())  
        else:
            ep_data['act'].append(actions[0].cpu().numpy())
            ep_data['obs'].append(obs['obs']['state'][0].cpu().numpy())
            ep_data['rews'].append(reward[0].cpu().numpy())
        #ep_data['info'].append(info)
        x_pos = obs['obs']['state'][:,6].cpu().numpy()
        y_pos = obs['obs']['state'][:,7].cpu().numpy()
        ep_data['all_dist'].append(np.linalg.norm(np.array([x_pos, y_pos]), axis=0))
        total_reward += reward[0]
        num_steps += 1
        is_done = done.any()
    ep_data['obs'] = np.array(ep_data['obs'])
    ep_data['act'] = np.array(ep_data['act'])
    ep_data['rews'] = np.array(ep_data['rews'])
    ep_data['all_dist'] = np.array(ep_data['all_dist'])

    print(f'\n Episode: rew_sum={total_reward:.2f}, tot_steps={num_steps} \n')
    #print(f'Episode data: {ep_data} \n')
    print(f'Episode data obs shape: {ep_data["obs"].shape} \n')

    #if not cfg.headless:
    #plot_episode_data_virtual(ep_data, evaluation_dir, store_all_agents)
    success_rate = success_rate_from_distances(ep_data['all_dist'])
    print(success_rate)

In [7]:

# setting up the Isaac Gym environment and player

env_conf = models[0].split("/")
config_name = os.path.join(env_conf[0],env_conf[1],env_conf[2], "config.yaml")

config_train = "../cfg/train/MFP2D_PPOmulti_dict_MLP.yaml"
with open(config_train, 'r') as stream:
    cfg_train = yaml.safe_load(stream)
    
try:
    with open(config_name, 'r') as stream:
        cfg = yaml.safe_load(stream)
except:
    print("Error opening config file")
    exit()
    
# _____Set up task_____
horizon = 500
cfg['task']['env']['maxEpisodeLength'] = horizon + 2
cfg['task']['env']['platform']['core']['mass'] = 5.32
cfg['task']['env']['clipObservations']['state'] = 20.0
cfg['task']['env']['task_parameters']['max_spawn_dist'] = 3.0
cfg['task']['env']['task_parameters']['min_spawn_dist'] = 1.5  
cfg['task']['env']['task_parameters']['kill_dist'] = 6.0
cfg['task']['env']['task_parameters']['kill_after_n_steps_in_tolerance'] = 800
cfg['train'] = cfg_train
cfg_dict = omegaconf_to_dict(cfg)
# _____Create environment_____
headless = True
enable_viewport = False
env = VecEnvRLGames(headless=headless, sim_device=cfg['device_id'], enable_livestream=False, enable_viewport=enable_viewport)

from omni.isaac.core.utils.torch.maths import set_seed
cfg['seed'] = set_seed(cfg['seed'], torch_deterministic=cfg['torch_deterministic'])
cfg_dict['seed'] = cfg['seed']
print(cfg_dict)
task = initialize_task(cfg_dict, env)
rlg_trainer = RLGTrainer(cfg, cfg_dict)
rlg_trainer.launch_rlg_hydra(env)
rlg_config_dict = omegaconf_to_dict(cfg['train'])

# _____Create players (model)_____
runner = Runner(RLGPUAlgoObserver())
runner.load(rlg_config_dict)
runner.reset()

agent = runner.create_player()

eval_multi_agents(agent, models, horizon)

env.close()

Starting kit application with the following args:  ['/home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/apps/omni.isaac.sim.python.gym.headless.kit', '--/app/tokens/exe-path=/home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=True', '--ext-folder', '/home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/exts', '--ext-folder', '/home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/matteo/.local/sh

/home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/kit/exts/omni.graph/omni/graph/core/_impl/autonode/type_definitions.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Float = NewType("Float", np.float)


s a PCIe pass through for virtual machines.
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin] 
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin] -----------------------------------------------------------------------
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin] !!!!! Local system validation failed! Incorrect configuration detected.
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin]       Summary below. Details above.
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin] -----------------------------------------------------------------------
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin] 
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin]  - ECC:                         OK
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin]  - IOMMU:                       FAILED
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin] 
2023-07-11 12:53:45 s] [Warning] [gpu.foundation.plugin] ---------------------------------------------------------

/home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/kit/kernel/py/omni/kit/app/_impl/__init__.py:104: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(setting_py, str) and isinstance(setting_py, collections.Sequence):


Warp 0.6.3 initialized:
   CUDA Toolkit: 11.5, Driver: 12.0
   Devices:
     "cpu"    | x86_64
     "cuda:0" | NVIDIA RTX A2000 Laptop GPU (sm_86)
   Kernel cache: /home/matteo/.cache/warp/0.6.3
ar-2.0.5] startup
[2.196s] [ext: omni.kit.widget.filebrowser-2.3.10] startup
[2.202s] [ext: omni.kit.widget.versioning-1.3.8] startup
[2.205s] [ext: omni.kit.notification_manager-1.0.5] startup
[2.209s] [ext: omni.iray.libs-0.0.0] startup
[2.217s] [ext: omni.kit.window.filepicker-2.7.15] startup
[2.276s] [ext: omni.ui_query-1.1.1] startup
[2.277s] [ext: omni.mdl.neuraylib-0.1.0] startup
[2.282s] [ext: omni.kit.window.file_importer-1.0.10] startup
[2.284s] [ext: omni.kit.ui_test-1.2.9] startup
[2.289s] [ext: omni.mdl-0.1.0] startup
[2.311s] [ext: omni.kit.widget.zoombar-1.0.4] startup
[2.313s] [ext: omni.kit.widget.searchfield-1.0.10] startup
[2.315s] [ext: omni.kit.material.library-1.3.21] startup
[2.322s] [ext: omni.kit.browser.core-2.2.2] startup
[2.329s] [ext: omni.kit.window.file_exporter-1

2023-07-11 12:53:47 [2,876ms] [Error] [carb.events.python] RuntimeError: Cannot run the event loop while another loop is running

At:
  /home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/kit/python/lib/python3.7/asyncio/base_events.py(526): _check_runnung
  /home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/kit/python/lib/python3.7/asyncio/base_events.py(531): run_forever
  /home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/kit/extscore/omni.kit.async_engine/omni/kit/async_engine/async_engine.py(77): _on_update
  /home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py(197): __init__
  /home/matteo/.local/share/ov/pkg/isaac_sim-2022.2.1/exts/omni.isaac.gym/omni/isaac/gym/vec_env/vec_env_base.py(45): __init__
  /tmp/ipykernel_98301/3433912710.py(31): <module>
  /home/matteo/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3553): run_code
  /home/matteo/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.p

KeyError: '${task.name}'

### Code for inference only Player

config_name = "../cfg/train/MFP2D_PPOmulti_dict_MLP.yaml"
with open(config_name, 'r') as stream:
    cfg = yaml.safe_load(stream)
obs_space = spaces.Dict({"state":spaces.Box(np.ones(10) * -np.Inf, np.ones(10) * np.Inf),
                         "transforms":spaces.Box(low=-1, high=1, shape=(8, 5)),
                         "masks":spaces.Box(low=0, high=1, shape=(8,))})
act_space = spaces.Tuple([spaces.Discrete(2)]*8)
player = BasicPpoPlayerDiscrete(cfg, obs_space, act_space, clip_actions=False, deterministic=True)
model_path = "../corl_runs/MLP_GTXY_UF_0.25_ST_PE_0.01_PAV_1.0_PLV_0.01/nn/last_MLP_GTXY_UF_0.25_ST_PE_0.01_PAV_1.0_PLV_0.01_ep_2000_rew_589.91455.pth"
player.restore(model_path)

obs = dict({'state':torch.zeros((1,10), dtype=torch.float32, device='cuda'),
            'transforms': torch.zeros(5,8, device='cuda'),
            'masks': torch.zeros(8, dtype=torch.float32, device='cuda')})

action = player.get_action(obs, is_deterministic=True)

print(action)